# Preparations

In [7]:
# imports
import os
import glob
import pandas as pd

# parameters
wkdir="/data/CARD/projects/prs_prog"
prs_ref=f'{wkdir}/PD_meta_analysis_PDGENE_PDWBS/PD_meta_analysis_PDGENE_PDWBS.txt' 
pathway_folder=f'{wkdir}/pathways_extract_hg19' # provided by manuela
prsfolder=f'{wkdir}/prs'
resfolder=f'{wkdir}/report'
exclusion_regions='/data/CARD/GENERAL/exclusion_regions_hg19.txt' # stored in the common resource

plink2='/data/CARD/PD/AMP-PD/Plink/plink2_dev_June8/plink2'
prsice2='/data/CARD/projects/prs_prog/PRSice_linux'

pathways=['endocytosis', 'adaptive_immune', 'innate_immune', 'lysosome', 
          'mitochondria', 'microglia', 'monocytes', 'alpha_synuclein']

datasets=['PRECEPT', 'DATATOP','CORIELL', 'DIGPD_chip', 'DIGPD_neuroX',
          'PICNICS', 'SCOPA', 'PARKWEST', 'PARKFIT']

cohort_dic = {'PRECEPT':'PreCEPT_PostCEPT', 'DATATOP':'DATATOP', 
              'CORIELL':'NET_PD_LS1', 'PICNICS':'PICNICS', 
              'DIGPD_chip':'DIGPD', 'DIGPD_neuroX':'DIGPD', 
              'SCOPA':'PROPARK', 'PARKWEST':'PARKWEST', 'PARKFIT':'PARKFIT'}

outcome_dic={'AAO':'cs', 'pRBD':'cs',
             'MDS_UPDRS2':'lt', 'MDS_UPDRS3':'lt', 'MOCA':'lt',
             'HY3':'surv', 'MCI':'surv'}

In [8]:
ls {pathway_folder}

adaptive_immune_extract.txt  lysosome_extract.txt
alpha_synuclein_extract.txt  microglia_extract.txt
endocytosis_extract.txt      mitochondria_extract.txt
innate_immune_extract.txt    monocytes_extract.txt


In [9]:
!ls {wkdir}/PD_meta_analysis_PDGENE_PDWBS/

PD_meta_analysis_PDGENE_PDWBS.txt  Readme.txt


In [10]:
ref = pd.read_csv(prs_ref, sep = "\t", engine='c')
ref['SNP'] = ref.CHR.astype('str') + ':' + ref.BP.astype('str')
ref['P'] = ref['P.META']
ref['OR'] = ref['OR.META']
ref[['CHR', 'BP', 'A1', 'A2', 'SNP', 'P', 'OR']].to_csv(f'{wkdir}/prs_base.txt', sep='\t',
                                                        index=False)
!head {wkdir}/prs_base.txt

CHR	BP	A1	A2	SNP	P	OR
4	15753299	G	A	4:15753299	2.27e-08	0.9059999999999999
4	80878580	C	T	4:80878580	0.0137	0.9670000000000001
4	77269262	G	A	4:77269262	8.809999999999999e-09	0.93
4	90643051	A	C	4:90643051	1.46e-42	1.344
4	77147513	A	G	4:77147513	7.52e-09	1.11
4	15694890	G	A	4:15694890	1.26e-05	0.9490000000000001
4	90448417	A	C	4:90448417	3.14e-06	1.068
4	941017	T	C	4:941017	1.73e-14	0.9079999999999999
4	90833421	G	A	4:90833421	1.67e-08	0.9309999999999999


# Create pathway specific PRS
Using createPRS.py

In [11]:
with open ('createPRS.swarm', 'w') as f:
    for pathway in pathways:
        for dataset in datasets:
            f.write(f'python3 createPRS.py --pathway {pathway} --dataset {dataset}\n')

In [12]:
!swarm -f createPRS.swarm --time=1:00:00 -g 10 -p 2 -b 1 --logdir swarm --module=R/4.1.0 --partition=norm  --devel # 21357529 # quick cannot use the plink2 AVX version

Loading modules R/4.1.0
72 commands run in 36 subjobs, each command requiring 10 gb and 1 thread, packing 2 processes simultaneously per subjob, allocating 36 cores and 72 cpus
sbatch --array=0-35 --job-name="swarm" --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=20480 --partition=norm --time=60:00 /spin1/swarm/iwakih2/rP9igKG3A9/swarm.batch


In [6]:
# !jobhist 21357529

# Standardize the PRS
PRS with the largest threshold among p<0.05 were used for the analysis. The PRS were normalized.


In [24]:
t=pd.DataFrame(columns=[
    'p_threshold_closest_to_0.05', 'n_snps', 'pathway', 'dataset'
])
for pathway in pathways:
    for dataset in datasets:
        prsfile = f'{wkdir}/temp/{pathway}/{dataset}/res.prsice'
        if os.path.exists(prsfile):
            d = pd.read_csv(prsfile, sep='\t')
            pthres=max(d.Threshold[d.Threshold<0.05])
            nsnp = d.loc[d.Threshold==pthres,'Num_SNP'].values[0]
            t.loc[len(t),:]=[pthres, nsnp, pathway, dataset]

            pthres_format=format(pthres, '.12g')
            cname=f'Pt_{pthres_format}'
            d = pd.read_csv(f'{wkdir}/temp/{pathway}/{dataset}/res.all_score',
                            delim_whitespace=True)
            x = d[cname]
            d['PRS_z'] = (x - x.mean())/d[cname].std()
            d[['IID', 'PRS_z']].to_csv(f'{prsfolder}/{pathway}.{dataset}.csv',
                                       index=False)
        else:
            print(f'[{pathway:15s}] specific PRS unavailable for [{dataset:15s}]')


In [26]:
t.to_csv(f'{resfolder}/n_snps_in_pathwayPRS_summary.tab', sep='\t', index=False)

# Analyze against outcomes

Use analyzePRS.R

In [27]:
# !ls /data/CARD/projects/prs_prog/report/
# !rm /data/CARD/projects/prs_prog/report/lt.csv /data/CARD/projects/prs_prog/report/cs.csv /data/CARD/projects/prs_prog/report/surv.csv

In [28]:
# !Rscript --vanilla analyzePRS.R PRECEPT PreCEPT_PostCEPT endocytosis AAO cs
# !Rscript --vanilla analyzePRS.R PRECEPT PreCEPT_PostCEPT endocytosis MDS_UPDRS3 lt
# !Rscript --vanilla analyzePRS.R PRECEPT PreCEPT_PostCEPT endocytosis HY3 surv

In [29]:
with open ('analyzePRS.swarm', 'w') as f:
    for pathway in pathways:
        for dataset, cohort in cohort_dic.items():
            for outcome, model in outcome_dic.items():
                f.write(f'Rscript --vanilla analyzePRS.R {dataset} {cohort} {pathway} {outcome} {model}\n')

In [30]:
!swarm -f analyzePRS.swarm --time=0:02:00 -g 5 -p 2 -b 40 --logdir swarm --module=R/4.1.0 --partition=quick --devel # 21419977

Loading modules R/4.1.0
504 commands run in 7 subjobs, each command requiring 5 gb and 1 thread, packing 2 processes simultaneously per subjob, running 40 processes serially per subjob, allocating 7 cores and 14 cpus
sbatch --array=0-6 --job-name="swarm" --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=10240 --partition=quick --time=01:20:00 /spin1/swarm/iwakih2/GevcDplvBV/swarm.batch


In [31]:
# !jobhist 21419977

In [32]:
import pandas as pd
cs=pd.read_csv(f'{resfolder}/cs.csv')
lt=pd.read_csv(f'{resfolder}/lt.csv')
sv=pd.read_csv(f'{resfolder}/surv.csv')

# Explanation of report folders
* n_snps_in_pathwayPRS_summary.tab: summary for the created PRS
* results of the pathway specific PRS analysis (separated by used models) --> cs.csv, lt.csv, surv.csv

In [33]:
!ls /data/CARD/projects/prs_prog/report/

cs.csv	lt.csv	n_snps_in_pathwayPRS_summary.tab  surv.csv


In [34]:
# !rm /data/CARD/projects/prs_prog/temp

In [36]:
!rm -rf swarm